# Bigram Language Model

In this lab we will implement a bigram language model and use it to compute the probability of some sample sentences.

As you go through, make sure you understand what's going on in each cell, and ask if it is unclear.

### Outcomes
* Know how to count word frequencies in a corpus using Python libraries.
* Understand how to compute conditional probabilities.
* Be able to apply the chain rule to compute the probability of a sentence.

### Overview

The first part of the notebook loads the same dataset as last week.
The next part splits the data into training and test sets, and tokenises the utterances.
After this there are some tasks to complete to implement and test the language model. 

# 1. Preparing the Data

In [1]:
from datasets import load_dataset

split = "train"
cache_dir = "./data_cache"

dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Generating train split:   0%|          | 0/3474 [00:00<?, ? examples/s]

OSError: Cannot find data file. 
Original error:
[Errno 2] No such file or directory: 'data_cache/downloads/extracted/16042defdbe73da99c5a717927e4933cf58bac8a078b8269d50d7d0ef95f458e/doc2dial/v1.0.1/doc2dial_dial_train.json'

In [ ]:
# Collect all the utterances into a list. 
# For this task, we don't care about the order of the utterances in the conversation -- 
# we will just be using the utterances of examples of the language we want to model.

utterances = []
for sample in dataset:
    turns = sample['turns']
    for turn in turns:
        if turn['role'] == 'user':
            utterances.append(turn['utterance'])
            
###
print(f'Number of utterances: {len(utterances)}')

In [ ]:
sample

In [ ]:
# Tokenise the samples. You can replace NLTK with another tokenizer if you prefer. 
import nltk

for i in range(len(utterances)):
    utterances[i] = nltk.word_tokenize(utterances[i])
    
print(utterances[2])

In [ ]:
# We need to put in some artificial start <s> and end <e> tokens. 
# These will be used to compute conditional probabilities of each word at the start of a sentence, 
# and conditional probabilities of ending the sentence after a particular word. 
# This lets us model which words are most likely to start or end a sentence. 

for i in range(len(utterances)):
    utterances[i] = ['<s>'] + utterances[i] + ['<e>']

In [ ]:
# Split the data into training and test using scikit-learn.
from sklearn.model_selection import train_test_split

train_size = 0.8
test_size = 0.2

# Split the train data from the test data
train_data, test_data = train_test_split(utterances, train_size=train_size, test_size=test_size)


print(f'The training set has {len(train_data)} samples and the test set has {len(test_data)} samples.')

# 2. Counting Tokens

The bigram language model needs to compute two sets of counts from the training data:
1. The counts of how many times each bigram occurs.
2. The counts of how many times each word type occurs as the first token in a bigram. 

Let's start by finding the vocabulary of unique token 'types': 

In [ ]:
import numpy as np

vocab = np.unique(np.concatenate(train_data))
V = len(vocab)

print(vocab)
print(f'There are {V} types in our vocabulary.')

# Currently, vocab is a numpy array. It may be simpler to work with list:
vocab = vocab.tolist()

Now we create an object to store the bigram counts in:

In [ ]:
# A matrix where row indexes will correspond to the first token in a bigram, 
# and column indexes to the second token. The indexes must map to the index
# of the token in the vocabulary. The values in the matrix will be the counts.
counts = np.ones((V, V))  # set to ones for add one smoothing

In [ ]:
# Here is an example of how to find the index of a given word:

word = '<s>'  # example word

def get_index_for_word(word, vocab):
    if word in vocab:
        index = vocab.index(word)
        # np.argwhere(vocab == word)[0][0]  # if vocab is a numpy array, we can find the index of the word like this
    else:
        index = -1
    return index

index = get_index_for_word(word, vocab)

print(index)

**TODO 2.1:** count the bigrams that occur in the training set.

In [10]:
for sen in train_data:
    ### WRITE YOUR CODE HERE
    for i in range(len(sen) - 1):
        token_1 = sen[i]
        token_2 = sen[i+1]
        
        index_1 = vocab.index(token_1)  # np.argwhere(vocab == token_1)[0][0]
        index_2 = vocab.index(token_2)  # np.argwhere(vocab == token_2)[0][0]
        
        counts[index_1, index_2] += 1


**TODO 2.2:** Apply numpy's sum() function to the 'counts' variable to compute the number of times each word type occurs as the first token in a bigram.

In [11]:
### WRITE YOUR CODE HERE
first_token_counts = np.sum(counts, axis=1)

**TODO 2.3:** Compute a matrix (numpy array) of conditional probabilities using the counts. Compute the log of this matrix as a variable 'log_cond_probs'.

In [12]:
### WRITE YOUR CODE HERE
cond_probs = counts / first_token_counts[:, None]
log_cond_probs = np.log(cond_probs)
print(cond_probs)
print(log_cond_probs)

[[0.00085121 0.00017024 0.00017024 ... 0.00017024 0.00017024 0.00017024]
 [0.00017209 0.00017209 0.00017209 ... 0.00017209 0.00017209 0.00017209]
 [0.00017173 0.00017173 0.00017173 ... 0.00017173 0.00017173 0.00017173]
 ...
 [0.00017209 0.00017209 0.00017209 ... 0.00017209 0.00017209 0.00017209]
 [0.00017212 0.00017212 0.00017212 ... 0.00017212 0.00017212 0.00017212]
 [0.00017212 0.00017212 0.00017212 ... 0.00017212 0.00017212 0.00017212]]
[[-7.0688532  -8.67829111 -8.67829111 ... -8.67829111 -8.67829111
  -8.67829111]
 [-8.66750795 -8.66750795 -8.66750795 ... -8.66750795 -8.66750795
  -8.66750795]
 [-8.66957087 -8.66957087 -8.66957087 ... -8.66957087 -8.66957087
  -8.66957087]
 ...
 [-8.66750795 -8.66750795 -8.66750795 ... -8.66750795 -8.66750795
  -8.66750795]
 [-8.66733585 -8.66733585 -8.66733585 ... -8.66733585 -8.66733585
  -8.66733585]
 [-8.66733585 -8.66733585 -8.66733585 ... -8.66733585 -8.66733585
  -8.66733585]]


**TODO 2.4:** Write a function that uses log_cond_probs to compute the probability of a given tokenised sentence, such as the example below.

In [13]:
# example tokenised sentence
sen = ['<s>', 'If', 'you', 'give', 'me', 'the', 'help', ',', 'what', 'is', 'the', 'payment', 'system', '?', '<e>']

In [14]:
def compute_log_prob_sen(sen, vocab, log_cond_probs):
    # WRITE YOUR CODE HERE
    log_prob_sen = 0
    for i in range(1, len(sen)):
        index_0 = get_index_for_word(sen[i-1], vocab)
        index_1 = get_index_for_word(sen[i], vocab)
        
        if index_0 == -1 or index_1 == -1:
            continue  # handle unknown words
            
        log_prob_sen += log_cond_probs[index_0, index_1]

    return log_prob_sen

log_prob_sen = compute_log_prob_sen(sen, vocab, log_cond_probs)
print(np.exp(log_prob_sen))

3.619475216769052e-30


**TODO 2.5:** Compute the perplexity over the whole test set. You will need to make sure your code can handle unknown words -- make sure it does not end up misusing the index of -1 returned by get_index_for_word() for unknown words.

In [15]:
def perplexity(sen, vocab, log_cond_probs):
    """
    Compute perplexity for one sentence
    """
    ### WRITE YOUR CODE HERE
    log_prob = compute_log_prob_sen(sen, vocab, log_cond_probs)
    ppl = np.exp(-log_prob * 1.0 / len(sen))  # perplexity of the sentence
    ###
    return ppl

# Use perplexity() function to compute average perplexity across whole test dataset
### WRITE YOUR CODE HERE
total = 0
for test_sen in test_data:
    ppl = perplexity(test_sen, vocab, log_cond_probs)
    total += ppl

avg_ppl = total / len(test_data)  # mean across all test sentences
###
print(avg_ppl)

147.68326880114358


**EXTENSION 1:** Use the language model to generate new sentences by sampling. 
You can follow the example below to sample using scipy's multinomial class. Replace the distribution with the conditional distribution we computed earlier.

In [16]:
from scipy.stats import multinomial

example_vocab = np.array(['a', 'b', 'c', 'd'])

distribution = [0.3, 0.2, 0.1, 0.4]
sample = multinomial.rvs(1, distribution)
sample_bool = sample.astype(bool)  # convert the sample from integer to boolean
generated_word = example_vocab[sample_bool][0]  # use the one-hot boolean vector to look up the word

print(generated_word)

b


In [17]:
current_tok = '<s>'
tokens = [current_tok]

while current_tok != '<e>' and len(tokens) < 1000:
    ### WRITE YOUR CODE HERE
    distribution = np.exp(log_cond_probs[get_index_for_word(current_tok, vocab), :])
    distribution = distribution / np.sum(distribution)  # normalise because numerical errors mean the values don't  quite sum to one

    sample = multinomial.rvs(1, distribution)  # sample one word conditioned on current_tok
    sample = np.where(sample)[0][0]
    current_tok = vocab[sample]
    
    ###
    tokens.append(current_tok)

print(tokens)
print(len(tokens))


['<s>', 'noteworthy', 'discount', 'Do', 'offenses', 'facilities', 'theme', 'reitrement', 'thats', 'gave', 'takes', '21', 'courts', 'stop', 'subvención', 'Washington', 'miss', 'g', 'full', 'internship', 'we', 'Intrastatal', 'class', 'professionals', 'mmmmh', 'graduate/professional', 'regarding', 'verified', 'labor', 'open', 'GE', 'special', 'Rules', 'Recharge', 'earn', 'deaf', 'liens', 'let', 'borrowers', 'registrant', 'doesnt', 'unexpired', 'D/E', 'troublesome', 'so', 'costly', 'recertifying', 'We', 'Hope', 'June', 'Assessments', 'exactly', 'our', 'exact', 'limousine', 'volunteer', 'labor', 'doubt', 'handicap', "'ve", 'sue', 'care', 'bullet', 'decision', 'Apply', '623', 'family', 'equipment', 'Eligibility', 'France', 'married', 'source', 'honors', 'un', 'Exit', 'services', 'react', 'release', 'areas', 'non-citizen', 'earned', 'texting', 'minor', 'taeaching', '20', 'Register', 'PIRP', 'Look', 'specially', 'started', '866', 'value', 'policy', 'false', 'following', 'temporary', 'February'

MORE EXTENSIONS: 
* Add some smoothing to the counts and see how it affects the results.
* Use trigrams instead of bigrams. Does it improve perplexity?